<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/3_submit_test/gp-submit_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Submit Test

제출 형태도 정해져있으므로 Validation Set을 이용해서 제출해보기

In [ ]:
# tutorial/3_submit_test/gp-submit_test.ipynb